In [ ]:
import sys
!{sys.executable} -m pip install facets-overview plotly scikit-learn

In [ ]:
import numpy as np
from collections import Counter

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sklearn

# CLUSTERING: K-Means

The data used for this tutorial comes from Tiller, a cash register software acquired by sumup recently.

This is a csv where each line represents an order (or orders) from a restaurant (or store). An order is the equivalent of a table order. Several people can therefore pay the final bill.

For each restaurant, a maximum of 20k orders were randomly selected to limit the final size of the dataset.

The objective of this tutorial is to:
- manipulate the dataset by constructing variables
- apply a clustering algorithm to the variables constructed beforehand in order to see if similar restaurants can be grouped together

## I. Data manipulation

### a) Loading the dataset
Download the data from this lab at [this address](https://drive.google.com/file/d/1turTzNTSjrUvmf-DqjFTqzblLPksuktm/view?usp=sharing)

In [ ]:
df = pd.read_csv('Tiller_order_data_clustering.csv')
print(df.shape)
df.head()

**Formatting**
- convert 'id_store' and 'id_order' fields to string
- format the date fields 'date_opened' and 'date_closed'
- 'is_open_weekend': boolean to know if the order was made during the weekend or not
- 'duration_opening': opening time range in seconds (difference between 'date_closed' and 'date_opened')

In [ ]:
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S UTC'

# convert to string
df['id_store'] = "store_" + df['id_store'].astype('str')
df['id_order'] = "order_" + df['id_order'].astype('str')

# date formatting
df['date_opened'] = pd.to_datetime(df['date_opened'], format=DATETIME_FORMAT)
df['date_closed'] = pd.to_datetime(df['date_closed'], format=DATETIME_FORMAT)

# is_open_weekend
df['is_open_weekend'] = df['date_opened'].dt.weekday.apply(lambda x: True if x in [5, 6] else False) # 5 and 6 for Saturday and Sunday

#duration_opening
df['duration_opening'] = df['date_closed'] - df['date_opened']
df['duration_opening'] = df['duration_opening'].dt.total_seconds()

### b) Construction of variables on a dataframe centered on restaurants

**Build a dataframe with one line per restaurant**

Hint: create a new dataframe 'df_by_store' with a groupby on the id_store, the id of the restaurant. We will then add columns for each variable associated with a restaurant.

In [ ]:
dfg = df.groupby('id_store')
df_by_store = pd.DataFrame(index=dfg.indices.keys())
df_by_store.head()

**It is on these variables that we are going to cluster the restaurants:**
- 'mean_m_cached_paid': average basket per customer
- 'mean_m_nb_customer': average number of customers per order
- 'percent_nb_orders_away': percentage of orders placed to go
- 'mean_duration_opening': average time spent at the table
- 'is_open_weekend': boolean to know if the restaurant is open on weekends or not

In [ ]:
# mean_m_cached_payed
df_by_store['sum_m_cached_payed'] = dfg['m_cached_payed'].sum().values
df_by_store['sum_m_nb_customer'] = dfg['m_nb_customer'].sum().values + 1
df_by_store['mean_m_cached_payed'] = df_by_store.apply(lambda row: row['sum_m_cached_payed'] / row['sum_m_nb_customer'], axis=1)

# mean_m_nb_customer
df_by_store['count_orders'] = dfg['id_order'].count().values
df_by_store['mean_m_nb_customer'] =  df_by_store.apply(lambda row: row['sum_m_nb_customer'] / row['count_orders'], axis=1)

# nb_orders_away
df_by_store['is_take_away'] = dfg['is_take_away'].sum().values
df_by_store['percent_nb_orders_away'] = df_by_store['is_take_away'] / df_by_store['count_orders'] * 100

# mean_duration_opening
df_by_store['mean_duration_opening'] = dfg['duration_opening'].mean().values
df_by_store['mean_duration_opening'] = df_by_store['mean_duration_opening'].apply(lambda x: x if x > 0 else 0)

# is_open_weekend
df_by_store['is_open_in_weekend'] = dfg['is_open_weekend'].apply(list).apply(any)
df_by_store['is_open_in_weekend'] = df_by_store['is_open_in_weekend'].apply(lambda x: 1 if True else 0)

**Remove intermediate columns 'count_orders', 'sum_m_cached_paid' and 'is_take_away' with [drop](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)**

In [ ]:
# to be completed

**Display the first line of the final dataset with iloc**

In [ ]:
# to be completed

**Analyze the distribution of each of these variables with [Facets](https://pair-code.github.io/facets/)**

In [ ]:
# Create the feature stats for the datasets and stringify it.
import base64
from facets_overview.generic_feature_statistics_generator import GenericFeatureStatisticsGenerator

gfsg = GenericFeatureStatisticsGenerator()
proto = gfsg.ProtoFromDataFrames([{'name': 'all', 'table': df_by_store}])
protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")

# Display the facets overview visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """
        <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html" >
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display(HTML(html))

## II. Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

**Instantiation of variable 'X' as being a [copy](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html) of dataframe 'df_by_store'**

In [ ]:
# to be completed

**Apply clustering [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) with 3 clusters.**

In [ ]:
# to be completed

**Save the ids of the clusters found ('model.labels_') in a new column of the dataframe 'X' named 'cluster'.**

In [ ]:
# to be completed

**The following code ([TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)) creates two columns aggregating most of the information contained in the set of variables of the dataset. We keep here only two dimensions in order to be able to display the restaurants on a scatterplot in 2 dimensions.**

In [ ]:
X_emb = TSNE(n_components=2).fit_transform(X)
X_emb = pd.DataFrame(X_emb, columns=['d1', 'd2'], index=X.index)
X_emb

**Concatenate 'X' with 'X_emb' by concretely adding the two columns of 'X_emb' on 'X'. You will name the resulting dataset 'X_scatter'.**

In [ ]:
# to be completed

## III. Return of Plotly

**Example of using go.scatter instead of px.scatter**

'plotly.graph_objects' allows access to a lower level interface of Plotly, and therefore to further customize its graphs.

In this visualization you can see:
- the use of [subplot](https://plotly.com/python/subplots/)
- a loop on each group of a cluster, adding one layer per subplot (via 'fig.add_trace').
- the use of 'legendgroup' to synchronize clicks on the legend (the cluster(s) on the right).
- the display of the values ​​of the variables corresponding to the restaurants (at the points) when passing the mouse over the points (hover)
- saving in HTML

In [ ]:
# color management in order to always have the same color per cluster:
cluster_colors = {}
for i, cluster in enumerate(X_scatter['cluster'].unique()):
    i += 1
    cluster_colors[cluster] = px.colors.qualitative.Light24_r[i % len(px.colors.qualitative.Light24_r)]
len(cluster_colors)

In [ ]:
# creating a subplot
fig = make_subplots(
    rows=2,
    cols=1,
    row_heights=[0.75, 0.25],
    vertical_spacing=0.15,
    subplot_titles=[
        "Projection of restaurants by cluster",
        "Characterization of clusters"
    ]
)

# For each cluster, add a layer on the figure
for cluster, rows in X_scatter.groupby(['cluster']):

    # Projection of restaurants by cluster
    rows['info_selectors'] = rows.apply(lambda x: '<br>'.join([col + ' : ' + str(np.around(float(val), 2))
                                                               for col, val in zip(rows.columns, x)]), axis=1)
    fig.add_trace(
        go.Scatter(
            x=rows['d1'],
            y=rows['d2'],
            # formatting points:
            mode='markers',
            opacity=1,
            marker_symbol=1,
            # color management per cluster:
            marker_color=cluster_colors[cluster],
            # allows to synchronize the selection by cluster on the two graphs:
            legendgroup='group' + str(cluster),
            # display legend for selection by cluster:
            name='Cluster ' + str(cluster + 1),
            # hover:
            hovertemplate = "%{hovertext}",
            hovertext=list(rows['info_selectors'].values),
            hoverlabel=dict(namelength=0),
        ), row=1, col=1
    )

    # characterization of clusters
    features = rows. copy()
    features.drop(['cluster', 'd1', 'd2', 'info_selectors'], inplace=True, axis=1)
    fig.add_trace(
        go.Bar(
            x=features.columns,
            y=features.mean(axis=0),
            # color management per cluster:
            marker_color=cluster_colors[cluster],
            # barmode:
            base='stack',
            # allows to synchronize the selection by cluster on the two graphs:
            legendgroup='group' + str(cluster),
            # avoid displaying the selection twice per cluster:
            showlegend=False,
            # hover:
            hovertemplate = "Cluster: %{hovertext}<br> Variable: %{x}<br> Average value: %{y}",
            hovertext=[cluster + 1 for cluster in rows['cluster'].values],
            hoverlabel=dict(namelength=0),
        ), row=2, col=1
    )

# formatting
fig.update_xaxes(
    showgrid=False,
    visible=False,
    row=1,
    collar=1
)
fig.update_yaxes(
    showgrid=False,
    visible=False,
    row=1,
    collar=1
)
fig.update_yaxes(
    type='log',
    row=2,
    collar=1
)
fig.update_layout(
   paper_bgcolor='rgba(0,0,0,0)',
   plot_bgcolor='rgba(0,0,0,0)',
   caption=dict(
       title='Clusters',
       bgcolor='White',
       bordercolor="Black",
       borderwidth=2,
       font=dict(family="Courier", size=12, color="black"),
       title_font_family="Courier"
   ),
   height=800
)

fig.show()

Save this plot in interactive html via the command [fig.write_html](https://plotly.com/python/interactive-html-export/):

In [ ]:
# to be completed

# CLUSTERING: DBSCAN
This part makes it possible to apply unsupervised machine learning techniques in order to group places of interest (cluster) from geospatial data. Extracting places of interest then makes it possible to infer living areas or usual places.

Download the data at [this address](https://drive.google.com/file/d/1WcLvHOZMSa6Sa4gT4yNMUkrTuQOGNRdS/view?usp=sharing)

We will work here on the latitudes and longitudes of taxi ride requests in NY, which we will cluster into zones.

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
df = # to be completed
df.rename({'pickup_latitude': 'Latitude', 'pickup_longitude': 'Longitude'}, axis=1, inplace=True)
print(df.shape)
df.head()

**Remove rows if latitude or longitude is missing (NaN) using [dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna. html)**

In [ ]:
# to be completed

**Convert 'Latitude' and 'Longitude' columns to radians (via [np.radians](https://numpy.org/doc/stable/reference/generated/numpy.radians.html)). You will name the resulting columns 'Latitude_rad' and 'Longitude_rad'.**

In [ ]:
# to be completed

**The following hyperparameters allow you to specify a distance in meters as a clustering parameter**

In [ ]:
MAX_DIST_M = 100
MIN_SAMPLES = 10

EARTH_RADIUS=6371088 # meters

def m2rad(dist):
    return dist / EARTH_RADIUS

dbscan_args = {
    'eps': m2rad(MAX_DIST_M),
    'min_samples': MIN_SAMPLES,
    'metric': 'haversine'
}
dbscan_args

**Train the clustering algorithm [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) on the 'Latitude_rad' and 'Longitude_rad' columns**

In [ ]:
# to be completed

**Add a 'cluster' column containing the cluster ids.
Hint: use 'labels_'**

In [ ]:
# to be completed

**Display the number of points per cluster.**

In [ ]:
# to be completed

**Display the rows corresponding to cluster 12.**

In [ ]:
# to be completed

**Display the points via a scatter plot (with plotly), using latitudes and longitudes for the ordinate and abscissa axes, as well as the 'cluster' column for the color. Then zoom in on this graph. What do you notice?.**

In [ ]:
# to be completed

### BONUS:
- Vary the 'epsilon' and 'min_sample' parameters of the algorithm. What do you notice ? In your opinion, what are the limits of a DBSCAN approach?
- Test the OPTICS algorithm and compare